In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../envs'))
if module_path not in sys.path:
    sys.path.append(module_path)

from mobility.taxinet.env import TaxiNetEnv

In [6]:
import gym
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [7]:
env = TaxiNetEnv(file_path="../envs/mobility/resources/city.txt")
env.seed(123)
action_size = env.action_space.n
env.reset()

31850

In [8]:
model = Sequential()
model.add(Embedding(45000, 10, input_length=1))
model.add(Reshape((10,)))
model.add(Dense(4500, activation='relu'))
model.add(Dense(4500, activation='relu'))
model.add(Dense(4500, activation='relu'))
model.add(Dense(action_size, activation='linear'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             450000    
_________________________________________________________________
reshape (Reshape)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 4500)              49500     
_________________________________________________________________
dense_1 (Dense)              (None, 4500)              20254500  
_________________________________________________________________
dense_2 (Dense)              (None, 4500)              20254500  
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 27006     
Total params: 41,035,506
Trainable params: 41,035,506
Non-trainable params: 0
____________________________________________

In [9]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=action_size, memory=memory, nb_steps_warmup=500, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=1, nb_max_episode_steps=99, log_interval=100000)

In [ ]:
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

In [ ]:
dqn.test(env, nb_episodes=5 , visualize=False, nb_max_episode_steps=99)